In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.getcwd()
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLPP/Univ_NLP')
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/NLPP/Univ_NLP'

In [ ]:
ls

data.json  data_pre.ipynb  README.md  train_model.ipynb


In [ ]:
!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.6/754.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 55.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-k4dpk0d_/kobert-tokenizer_88753a2db913436e8a22f

In [ ]:
#오류발생으로 numpy downgrade
import numpy as np
np.__version__
import gluonnlp as nlp

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
import torch
torch.cuda.is_available()  # True가 나오면 GPU 사용 가능

True

In [ ]:
import pandas as pd

# JSON 파일 로드
file_path = '/content/drive/MyDrive/Colab Notebooks/NLPP/Univ_NLP/data.json'
df = pd.read_json(file_path, orient='records', lines=True)

# 데이터 확인
df.sample(5)

,info.date,info.gender,info.ageRange,info.experience,question.raw.text,answer.summary.text,emotion_text,emotion_expression,emotion_category,intent_text,intent_expression,intent_category
2791,20230116,MALE,-34,NEW,다른 계열사도 많이 있는데 이 계열사에 지원해 주신 특별한 이유가 있다면 그 이유를...,이 계열사에 지원한 이유는 회사 이름보다 계열사 무엇을 하는지가 더 중요했기 때문입...,[],[],[],[''],[''],['attitude']
338,20230116,FEMALE,-34,NEW,지원자분께서 최근에 공부하고 있는 것이 있으신가요 있으시다고 하면 왜 그걸 공부하시...,에스큐엘 구문을 이용한 데이터의 적재와 활용도를 보고 데이터 분석을 하는데 활용을 ...,[],[],[],"['네 요즘은 에스큐엘 문법에 관하여 공부하고 있습니다.', '그래서 지금은 이제 ...","['i_hwsw_prfi', 'i_hwsw_prfi']","['attitude', 'attitude']"
3233,20230116,MALE,-34,NEW,만약 본인이 당장 내일 죽는다는 사실을 알게 된다면 무엇을 하실 건가요 또 이 면접...,지금 면접을 보고 회사를 다니고 싶은 이유는 노년까지 산다 라는 네 커리큘럼 상으로...,[],[],[],[''],[''],['attitude']
722,20230116,FEMALE,45-54,NEW,요즘 공부하고 있는 것이 있으신가요 있다면 무엇을 공부하고 있고 왜 공부하고 있으며...,에스앤에스가 활성화되면서 수많은 데이터들을 어떻게 정제하고 유용한 정보로 분석을 하...,[],[],[],[''],[''],['attitude']
5484,20230116,MALE,-34,EXPERIENCED,지원자님은 팀으로 일하는 것과 혼자 일하는 것 중 무엇을 더 선호하시는지 이유와 함...,팀으로 일하는 것도 좋지만 혼자 일하는 쪽을 조금 더 선호하는 편입니다. 더 넓은 ...,[],[],[],[''],[''],['attitude']


In [ ]:
# info.date 삭제(모두 같은 날짜임)
df = df.drop(columns=['info.date'])

df.head(2)

,info.gender,info.ageRange,info.experience,question.raw.text,answer.summary.text,emotion_text,emotion_expression,emotion_category,intent_text,intent_expression,intent_category
0,FEMALE,-34,NEW,지원자님이 가장 존경하는 역사 속 인물을 말씀해 주십시오 그리고 그 인물을 왜 존경...,저는 성경에 나오는 인물인 모세를 개인적으로 가장 존경합니다. 본인의 능력을 최대한...,"['제가 역사 속에서 가장 존경하는 인물은 성경에 나오는 인물인 모세입니다.', '...","['p-affection', 'p-affection']","['positive', 'positive']",[''],[''],['attitude']
1,FEMALE,-34,NEW,지원자님께서 지금까지 경험하신 여러 대외 활동 중에 무엇을 배웠고 어떤 자세로 임했...,지금까지 했던 대외 활동들은 대부분 기업의 홍보 대사나 마케팅 활동을 많이 했습니다...,['지금까지 했던 대외 활동들은 대부분 기업의 홍보 대사나 마케팅 활동을 많이 했었...,['u-fact'],['neutral'],['그 활동을 하면서 이 같은 과에 비슷한 사람들과 어울리기보다는 조금 더 다양한 ...,"['c_cop', 'c_cop']","['personality', 'personality']"


In [ ]:
# import numpy as np

# # 리스트 평탄화 후 고유한 값 찾기
# flattened = np.concatenate(df['intent_category'].values)
# intent_category_unique_values = np.unique(flattened)

# # 리스트 평탄화 후 고유한 값 찾기
# flattened = np.concatenate(df['emotion_expression'].values)
# emotion_expression_unique_values = np.unique(flattened)

# # 리스트 평탄화 후 고유한 값 찾기
# flattened = np.concatenate(df['emotion_category'].values)
# emotion_category_unique_values = np.unique(flattened)

# # 리스트 평탄화 후 고유한 값 찾기
# flattened = np.concatenate(df['intent_expression'].values)
# intent_expression_unique_values = np.unique(flattened)

# print(intent_category_unique_values)
# print(intent_expression_unique_values)
# print(emotion_category_unique_values)
# print(emotion_expression_unique_values)

In [ ]:
import numpy as np

flattened = np.concatenate(df['intent_category'].values)
intent_category_unique_values = np.unique(flattened)
print(intent_category_unique_values)

["'attitude'" "'background'" "'etc'" "'personality'" "'technology'"]


In [ ]:
# 실무(직무) 면접 : "technology", "attitude"
# 인성 면접: "background", "personality" ,"etc"

# 맨 앞 카테고리만 가져와야함
df['category'] = df['intent_category'].apply(lambda x: x[0] if isinstance(x, list) else x)

df.sample()

,info.gender,info.ageRange,info.experience,question.raw.text,answer.summary.text,emotion_text,emotion_expression,emotion_category,intent_text,intent_expression,intent_category,category
4390,MALE,55-,NEW,해외에 파견 근무를 나가게 된다면 어느 나라에서 근무하고 싶으신가요 이유도 함께 말...,캐나다 밴쿠버에 선택권이 있다면 영어권이라 항상 일정한 기온을 가지는 지역인 밴쿠버...,[],[],[],[''],[''],['attitude'],'attitude'


In [ ]:
  # info.gender -> MALE=0, FEMALE=1
df['info.gender'] = df['info.gender'].map({'MALE': 0, 'FEMALE': 1})

# info.ageRange -> ['-34', '35-44', '55-', '45-54'] -> 0, 1, 2, 3
age_range_map = {'-34': 0, '35-44': 1, '45-54': 2, '55-': 3}
df['info.ageRange'] = df['info.ageRange'].map(age_range_map)

# info.experience -> ['NEW', 'EXPERIENCED'] -> 0, 1
df['info.experience'] = df['info.experience'].map({'NEW': 0, 'EXPERIENCED': 1})

In [ ]:
# 'category' 개수 구하기
# count_technology = df[df['category'] == '\'attitude\''].shape[0]
# print(count_technology)

# 실무(직무) 면접 : "technology" 150, "attitude 4991"
# 인성 면접: "background"421 , "personality180" ,"etc120"

In [ ]:
category_range_map = {'\'technology\'': 0, '\'attitude\'': 1, '\'background\'': 2, '\'personality\'': 3, '\'etc\'': 4}
df['category'] = df['category'].map(category_range_map)

df.head()

,info.gender,info.ageRange,info.experience,question.raw.text,answer.summary.text,emotion_text,emotion_expression,emotion_category,intent_text,intent_expression,intent_category,category
0,1,0,0,지원자님이 가장 존경하는 역사 속 인물을 말씀해 주십시오 그리고 그 인물을 왜 존경...,저는 성경에 나오는 인물인 모세를 개인적으로 가장 존경합니다. 본인의 능력을 최대한...,"['제가 역사 속에서 가장 존경하는 인물은 성경에 나오는 인물인 모세입니다.', '...","['p-affection', 'p-affection']","['positive', 'positive']",[''],[''],['attitude'],1
1,1,0,0,지원자님께서 지금까지 경험하신 여러 대외 활동 중에 무엇을 배웠고 어떤 자세로 임했...,지금까지 했던 대외 활동들은 대부분 기업의 홍보 대사나 마케팅 활동을 많이 했습니다...,['지금까지 했던 대외 활동들은 대부분 기업의 홍보 대사나 마케팅 활동을 많이 했었...,['u-fact'],['neutral'],['그 활동을 하면서 이 같은 과에 비슷한 사람들과 어울리기보다는 조금 더 다양한 ...,"['c_cop', 'c_cop']","['personality', 'personality']",3
2,1,0,0,언제부터 아이씨티에 관심이 있었는지 그리고 지금과 같은 아이씨티 역량을 갖추기 위해...,이 회사에 맞는 인재가 되기 위해 프로그래밍 언어를 공부했습니다. 고난과 역경이 많...,[],[],[],['하지만 프로그래밍 언어를 스스로 배우면서 독학하면서 재미를 느꼈고 저는 이 ...,['c_sincere_job'],['etc'],4
3,1,0,0,지원자님이 다른 지원자 분들에 비해서 가지고 있는 차별성이 있다면 혹은 장점을 어필...,아이씨티 분야에서 남들보다 호기심이 많고 새로운 기술을 습득하는 것이 빠른 게 장점...,[],[],[],[''],[''],['attitude'],1
4,1,0,0,지원자님에 장점과 단점을 말씀해 주시고 그 장점을 특화시키고 단점을 극복하기 위해 ...,저는 유연성이 강해서 어떤 상황에서도 잘 적응하고 인간관계에서도 유연성이 있습니다....,['개인적으로 이야기할 때는 밝고 명랑하다는 소리를 많이 듣지만 많은 사람들 앞...,['n-shame'],['negative'],"['저의 장점은 첫 번째로 유연성이 강합니다.', '하지만 저의 단점은 첫 번째로 ...","['c_person', 'c_person', 'c_person']","['background', 'background', 'background']",2


In [ ]:
# 일부 컬럼 제외 나머지만 가져옴 df_all

df_all = df[['info.gender',	'info.ageRange', 'info.experience',	'question.raw.text'	,'answer.summary.text', 'category']]
df_all.head()

,info.gender,info.ageRange,info.experience,question.raw.text,answer.summary.text,category
0,1,0,0,지원자님이 가장 존경하는 역사 속 인물을 말씀해 주십시오 그리고 그 인물을 왜 존경...,저는 성경에 나오는 인물인 모세를 개인적으로 가장 존경합니다. 본인의 능력을 최대한...,1
1,1,0,0,지원자님께서 지금까지 경험하신 여러 대외 활동 중에 무엇을 배웠고 어떤 자세로 임했...,지금까지 했던 대외 활동들은 대부분 기업의 홍보 대사나 마케팅 활동을 많이 했습니다...,3
2,1,0,0,언제부터 아이씨티에 관심이 있었는지 그리고 지금과 같은 아이씨티 역량을 갖추기 위해...,이 회사에 맞는 인재가 되기 위해 프로그래밍 언어를 공부했습니다. 고난과 역경이 많...,4
3,1,0,0,지원자님이 다른 지원자 분들에 비해서 가지고 있는 차별성이 있다면 혹은 장점을 어필...,아이씨티 분야에서 남들보다 호기심이 많고 새로운 기술을 습득하는 것이 빠른 게 장점...,1
4,1,0,0,지원자님에 장점과 단점을 말씀해 주시고 그 장점을 특화시키고 단점을 극복하기 위해 ...,저는 유연성이 강해서 어떤 상황에서도 잘 적응하고 인간관계에서도 유연성이 있습니다....,2


In [ ]:
# df_all DataFrame 사용: 훈련set 검증set나누기

from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(df_all, test_size=0.25, random_state=0)

print("Training set shape:", dataset_train.shape)
print("Validation set shape:", dataset_test.shape)


Training set shape: (4383, 6)
Validation set shape: (1461, 6)


In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
from torch.utils.data import Dataset

# ★
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

https://blog.naver.com/newyearchive/223097878715

In [ ]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:

# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj

In [ ]:

import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)


In [ ]:
print(dataset_train['category'].unique())

# 'category' 열이 숫자형으로 되어 있는지 확인
print(dataset_train['category'].dtype)

[1 4 2 0 3]
int64


In [ ]:
# df- > 리스트 형태로 변환
dataset_train = dataset_train.values.tolist()
dataset_test = dataset_test.values.tolist()

In [ ]:
# self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair
# int() 에러 나서 보니 idx 설정 잘못해서.. 예제 코드 참고하다가 오류남(df -> 리스트로 변환해야함)
data_train = BERTDataset(dataset_train, 3, 5, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 3, 5, tokenizer, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
import torch.nn as nn

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
# GPU 사용 시
device = torch.device("cuda:0")
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
from tqdm.notebook import tqdm  # tqdm_notebook 대신 tqdm.notebook을 사용

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/69 [00:00<?, ?it/s]

TypeError: dropout(): argument 'input' (position 1) must be Tensor, not str

In [ ]:
outputs = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))
print(outputs)

In [ ]:
print(type(token_ids), type(segment_ids), type(valid_length), type(label))  # 타입 확인


In [ ]:
"""

In [ ]:
#tokenize
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

data_train = BERTDataset(train_df, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(val_df, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(train_df, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(val_df, batch_size=batch_size, num_workers=5)

In [ ]:
"""""" 지피티코드

In [ ]:
  from torch.utils.data import Dataset
  import torch
  from transformers import BertTokenizer

  class BERTDataset(Dataset):
      def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad=True, pair=False):
          self.tokenizer = bert_tokenizer
          self.max_len = max_len
          self.pad = pad  # pad를 인스턴스 변수로 저장
          self.sentences = dataset[sent_idx].values  # 질문 텍스트
          self.labels = dataset[label_idx].values  # 레이블 (카테고리)

      def __getitem__(self, idx):
          text = self.sentences[idx]
          encoding = self.tokenizer.encode_plus(
              text,
              add_special_tokens=True,  # [CLS], [SEP] 토큰 추가
              max_length=self.max_len,  # 최대 길이
              padding="max_length" if self.pad else False,  # self.pad로 참조
              truncation=True,  # 자르기 처리
              return_tensors="pt"  # PyTorch tensor로 반환
          )

          input_ids = encoding['input_ids'].flatten()
          attention_mask = encoding['attention_mask'].flatten()
          # segment_ids는 한 문장만 처리할 때 0으로 설정
          segment_ids = torch.zeros_like(input_ids)
          label = torch.tensor(self.labels[idx], dtype=torch.long)

          return {
              'input_ids': input_ids,
              'attention_mask': attention_mask,
              'segment_ids': segment_ids,  # segment_ids 추가
              'labels': label
          }

      def __len__(self):
          return len(self.labels)

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader

# 데이터 ->bertdataset으로 변환
# BERTTokenizer 로드 (예: KoBERT를 사용)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # 모델에 맞는 토크나이저 사용

# BERTDataset으로 훈련 데이터와 검증 데이터를 변환
train_dataset = BERTDataset(train_df, sent_idx='question.raw.text', label_idx='category',
                            bert_tokenizer=bert_tokenizer, max_len=64, pad=True, pair=False)

val_dataset = BERTDataset(val_df, sent_idx='question.raw.text', label_idx='category',
                          bert_tokenizer=bert_tokenizer, max_len=64, pad=True, pair=False)

# 데이터로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
import torch.nn as nn

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
from torch.utils.data import DataLoader
# Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# tokenizer, model 정의
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
device = torch.device("cuda:0") #GPU 설정

#정의한 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

In [ ]:
from tqdm.notebook import tqdm
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

#1. 훈련루프
for epoch in range(num_epochs):  # num_epochs: 에폭 수
    model.train()  # 모델을 훈련 모드로 전환
    total_loss = 0
    total_accuracy = 0

    for batch_id, batch in enumerate(tqdm(train_dataloader)):  # train_dataloader에서 배치 처리
        optimizer.zero_grad()  # 이전 기울기 초기화

        token_ids = batch['input_ids'].long().to(device)
        segment_ids = batch['segment_ids'].long().to(device)  # segment_ids를 가져옴
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        valid_length = [len(seq) for seq in token_ids]

        # 모델 예측
        outputs = model(token_ids, valid_length, segment_ids)  # valid_length도 함께 전달

        # 손실 계산
        loss = loss_fn(outputs, labels)

        # 역전파
        loss.backward()
        optimizer.step()

        # 정확도 계산
        accuracy = calc_accuracy(outputs, labels)

        total_loss += loss.item()
        total_accuracy += accuracy

        if batch_id % log_interval == 0:  # log_interval에 맞춰 로그 출력
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_id+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}, Accuracy: {accuracy*100:.2f}%")

    # 에폭이 끝난 후 평균 손실과 정확도 출력
    avg_loss = total_loss / len(train_dataloader)
    avg_accuracy = total_accuracy / len(train_dataloader)
    print(f"Epoch {epoch+1} finished. Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy*100:.2f}%")

In [ ]:
# CPU로 데이터를 이동해 값 확인
print("token_ids:", batch['input_ids'].cpu())
print("segment_ids:", batch['segment_ids'].cpu())
print("attention_mask:", batch['attention_mask'].cpu())

In [ ]:
print("token_ids dtype:", batch['input_ids'].dtype)
print("segment_ids dtype:", batch['segment_ids'].dtype)
print("attention_mask dtype:", batch['attention_mask'].dtype)

In [ ]:
print("Batch size:", token_ids.size(0))  # 배치 크기
print("Sequence length:", token_ids.size(1))  # 시퀀스 길이

In [ ]:
# 배치 데이터 확인
print("Batch keys:", batch.keys())
print("input_ids shape:", batch['input_ids'].shape)
print("segment_ids shape:", batch['segment_ids'].shape)
print("attention_mask shape:", batch['attention_mask'].shape)

# 값 범위 및 고유 값 확인
print("Unique input_ids:", torch.unique(batch['input_ids']))
print("Unique segment_ids:", torch.unique(batch['segment_ids']))
print("Unique attention_mask:", torch.unique(batch['attention_mask']))

In [ ]:
print("Labels dtype:", batch['labels'].dtype)
print("Labels unique values:", torch.unique(batch['labels']))
print("Labels shape:", batch['labels'].shape)


In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
print("Model output size:", model(torch.randn(1, batch['input_ids'].shape[1]).to(device)).size())

In [ ]:
print(model)


In [ ]:
print("Input shape:", batch['input_ids'].shape)
print("Input data type:", batch['input_ids'].dtype)
print("Input value range:", batch['input_ids'].min(), batch['input_ids'].max())

In [ ]:
print("Original vocab size:", len(tokenizer))

In [ ]:
# 이미 자동으로 [CLS]와 [SEP]가 붙여진 데이터를 출력
print(train_df[['question.raw.text', 'answer.summary.text']].head())


데이터 불균형으로 인한 추가작업


In [ ]:
import nltk
#nltk.download('wordnet')

In [ ]:
import pandas as pd
import random
from nltk.corpus import wordnet

# 동의어 교체 함수
def synonym_replacement(sentence):
    words = sentence.split()
    new_words = words.copy()
    for i, word in enumerate(words):
        # 해당 단어의 동의어를 찾고, 하나를 선택
        synonyms = wordnet.synsets(word)
        if synonyms:
            synonym = random.choice(synonyms).lemmas()[0].name()
            new_words[i] = synonym
    return ' '.join(new_words)

# 카테고리별 증강 함수
def augment_data(df, category_column, text_columns, target_count):
    augmented_data = []

    # 카테고리별 처리
    for category, count in df[category_column].value_counts().items():
        print(f"\n[INFO] 현재 카테고리: {category}, 기존 데이터 개수: {count}")

        if count < target_count:
            required_count = target_count - count
            category_data = df[df[category_column] == category]

            # 증강 작업
            for i in range(required_count):
                row = category_data.sample(n=1).iloc[0]  # 랜덤 샘플 하나 선택
                augmented_row = row.copy()

                # 텍스트 컬럼에 대해 동의어 교체 수행
                for text_column in text_columns:
                    augmented_row[text_column] = synonym_replacement(row[text_column])

                augmented_data.append(augmented_row)

                # 진행 상황 출력
                if (i + 1) % 100 == 0:
                    print(f"\t{category}: {i + 1}/{required_count}개 증강 완료")

        # 카테고리 개수 출력
        print(f"\n[INFO] {category} 증강 완료 후 총 데이터 개수: {count + len(augmented_data)}")

    # 증강된 데이터프레임 반환
    return pd.DataFrame(augmented_data)

# 사용 예제
target_count = 5000  # 카테고리별 목표 개수
text_columns = ['question.raw.text', 'answer.summary.text']  # 증강 대상 컬럼

# 증강 실행
augmented_df = augment_data(df_combined, category_column='category', text_columns=text_columns, target_count=target_count)

# 증강된 데이터를 원본과 합치기
df_all_augmented = pd.concat([df_combined, augmented_df], ignore_index=True)

# 최종 카테고리별 데이터 개수 확인
print("\n[INFO] 최종 카테고리별 데이터 개수:")
print(df_all_augmented['category'].value_counts())

[INFO] 현재 카테고리: 1, 기존 데이터 개수: 5539

[INFO] 1 증강 완료 후 총 데이터 개수: 5539

[INFO] 현재 카테고리: 2, 기존 데이터 개수: 495
	2: 100/4505개 증강 완료
	2: 200/4505개 증강 완료
	2: 300/4505개 증강 완료
	2: 400/4505개 증강 완료
	2: 500/4505개 증강 완료
	2: 600/4505개 증강 완료
	2: 700/4505개 증강 완료
	2: 800/4505개 증강 완료
	2: 900/4505개 증강 완료
	2: 1000/4505개 증강 완료
	2: 1100/4505개 증강 완료
	2: 1200/4505개 증강 완료
	2: 1300/4505개 증강 완료
	2: 1400/4505개 증강 완료
	2: 1500/4505개 증강 완료
	2: 1600/4505개 증강 완료
	2: 1700/4505개 증강 완료
	2: 1800/4505개 증강 완료
	2: 1900/4505개 증강 완료
	2: 2000/4505개 증강 완료
	2: 2100/4505개 증강 완료
	2: 2200/4505개 증강 완료
	2: 2300/4505개 증강 완료
	2: 2400/4505개 증강 완료
	2: 2500/4505개 증강 완료
	2: 2600/4505개 증강 완료
	2: 2700/4505개 증강 완료
	2: 2800/4505개 증강 완료
	2: 2900/4505개 증강 완료
	2: 3000/4505개 증강 완료
	2: 3100/4505개 증강 완료
	2: 3200/4505개 증강 완료
	2: 3300/4505개 증강 완료
	2: 3400/4505개 증강 완료
	2: 3500/4505개 증강 완료
	2: 3600/4505개 증강 완료
	2: 3700/4505개 증강 완료
	2: 3800/4505개 증강 완료
	2: 3900/4505개 증강 완료
	2: 4000/4505개 증강 완료
	2: 4100/4505개 증강 완료
	2: 4200/4505개 증강 완료
	2: 4300/4505개 증강 완료
	2: 4400/4505개 증강 완료
	2: 4500/4505개 증강 완료

[INFO] 2 증강 완료 후 총 데이터 개수: 5000

[INFO] 현재 카테고리: 3, 기존 데이터 개수: 246
	3: 100/4754개 증강 완료
	3: 200/4754개 증강 완료
	3: 300/4754개 증강 완료
	3: 400/4754개 증강 완료
	3: 500/4754개 증강 완료
	3: 600/4754개 증강 완료
	3: 700/4754개 증강 완료
	3: 800/4754개 증강 완료
	3: 900/4754개 증강 완료
	3: 1000/4754개 증강 완료
	3: 1100/4754개 증강 완료
	3: 1200/4754개 증강 완료
	3: 1300/4754개 증강 완료
	3: 1400/4754개 증강 완료
	3: 1500/4754개 증강 완료
	3: 1600/4754개 증강 완료
	3: 1700/4754개 증강 완료
	3: 1800/4754개 증강 완료
	3: 1900/4754개 증강 완료
	3: 2000/4754개 증강 완료
	3: 2100/4754개 증강 완료
	3: 2200/4754개 증강 완료
	3: 2300/4754개 증강 완료
	3: 2400/4754개 증강 완료
	3: 2500/4754개 증강 완료
	3: 2600/4754개 증강 완료
	3: 2700/4754개 증강 완료
	3: 2800/4754개 증강 완료
	3: 2900/4754개 증강 완료
	3: 3000/4754개 증강 완료
	3: 3100/4754개 증강 완료
	3: 3200/4754개 증강 완료
	3: 3300/4754개 증강 완료
	3: 3400/4754개 증강 완료
	3: 3500/4754개 증강 완료
	3: 3600/4754개 증강 완료
	3: 3700/4754개 증강 완료
	3: 3800/4754개 증강 완료
	3: 3900/4754개 증강 완료
	3: 4000/4754개 증강 완료
	3: 4100/4754개 증강 완료
	3: 4200/4754개 증강 완료
	3: 4300/4754개 증강 완료
	3: 4400/4754개 증강 완료
	3: 4500/4754개 증강 완료
	3: 4600/4754개 증강 완료
	3: 4700/4754개 증강 완료

[INFO] 3 증강 완료 후 총 데이터 개수: 9505

[INFO] 현재 카테고리: 0, 기존 데이터 개수: 162
	0: 100/4838개 증강 완료
	0: 200/4838개 증강 완료
	0: 300/4838개 증강 완료
	0: 400/4838개 증강 완료
	0: 500/4838개 증강 완료
	0: 600/4838개 증강 완료
	0: 700/4838개 증강 완료
	0: 800/4838개 증강 완료
	0: 900/4838개 증강 완료
	0: 1000/4838개 증강 완료
	0: 1100/4838개 증강 완료
	0: 1200/4838개 증강 완료
	0: 1300/4838개 증강 완료
	0: 1400/4838개 증강 완료
	0: 1500/4838개 증강 완료
	0: 1600/4838개 증강 완료
	0: 1700/4838개 증강 완료
	0: 1800/4838개 증강 완료
	0: 1900/4838개 증강 완료
	0: 2000/4838개 증강 완료
	0: 2100/4838개 증강 완료
	0: 2200/4838개 증강 완료
	0: 2300/4838개 증강 완료
	0: 2400/4838개 증강 완료
	0: 2500/4838개 증강 완료
	0: 2600/4838개 증강 완료
	0: 2700/4838개 증강 완료
	0: 2800/4838개 증강 완료
	0: 2900/4838개 증강 완료
	0: 3000/4838개 증강 완료
	0: 3100/4838개 증강 완료
	0: 3200/4838개 증강 완료
	0: 3300/4838개 증강 완료
	0: 3400/4838개 증강 완료
	0: 3500/4838개 증강 완료
	0: 3600/4838개 증강 완료
	0: 3700/4838개 증강 완료
	0: 3800/4838개 증강 완료
	0: 3900/4838개 증강 완료
	0: 4000/4838개 증강 완료
	0: 4100/4838개 증강 완료
	0: 4200/4838개 증강 완료
	0: 4300/4838개 증강 완료
	0: 4400/4838개 증강 완료
	0: 4500/4838개 증강 완료
	0: 4600/4838개 증강 완료
	0: 4700/4838개 증강 완료
	0: 4800/4838개 증강 완료

[INFO] 0 증강 완료 후 총 데이터 개수: 14259

[INFO] 현재 카테고리: 4, 기존 데이터 개수: 129
	4: 100/4871개 증강 완료
	4: 200/4871개 증강 완료
	4: 300/4871개 증강 완료
	4: 400/4871개 증강 완료
	4: 500/4871개 증강 완료
	4: 600/4871개 증강 완료
	4: 700/4871개 증강 완료
	4: 800/4871개 증강 완료
	4: 900/4871개 증강 완료
	4: 1000/4871개 증강 완료
	4: 1100/4871개 증강 완료
	4: 1200/4871개 증강 완료
	4: 1300/4871개 증강 완료
	4: 1400/4871개 증강 완료
	4: 1500/4871개 증강 완료
	4: 1600/4871개 증강 완료
	4: 1700/4871개 증강 완료
	4: 1800/4871개 증강 완료
	4: 1900/4871개 증강 완료
	4: 2000/4871개 증강 완료
	4: 2100/4871개 증강 완료
	4: 2200/4871개 증강 완료
	4: 2300/4871개 증강 완료
	4: 2400/4871개 증강 완료
	4: 2500/4871개 증강 완료
	4: 2600/4871개 증강 완료
	4: 2700/4871개 증강 완료
	4: 2800/4871개 증강 완료
	4: 2900/4871개 증강 완료
	4: 3000/4871개 증강 완료
	4: 3100/4871개 증강 완료
	4: 3200/4871개 증강 완료
	4: 3300/4871개 증강 완료
	4: 3400/4871개 증강 완료
	4: 3500/4871개 증강 완료
	4: 3600/4871개 증강 완료
	4: 3700/4871개 증강 완료
	4: 3800/4871개 증강 완료
	4: 3900/4871개 증강 완료
	4: 4000/4871개 증강 완료
	4: 4100/4871개 증강 완료
	4: 4200/4871개 증강 완료
	4: 4300/4871개 증강 완료
	4: 4400/4871개 증강 완료
	4: 4500/4871개 증강 완료
	4: 4600/4871개 증강 완료
	4: 4700/4871개 증강 완료
	4: 4800/4871개 증강 완료

[INFO] 4 증강 완료 후 총 데이터 개수: 19097

[INFO] 최종 카테고리별 데이터 개수:
category
1    5539
4    5000
3    5000
0    5000
2    5000
Name: count, dtype: int64

In [ ]:
# 카테고리별 개수 확인
if 'category' in df_all_augmented.columns:
    category_counts = df_all_augmented['category'].value_counts()
    print("카테고리별 개수:")
    print(category_counts)
else:
    print("데이터프레임에 'category' 열이 없습니다.")

카테고리별 개수:
category
1    5539
4    5000
3    5000
0    5000
2    5000
Name: count, dtype: int64

In [ ]:
import pandas as pd
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize

# NLTK의 단어 토큰화 기능을 사용
#nltk.download('punkt')

# 예시 DataFrame (answer.summary.text 칼럼을 가정)
data = df_all_augmented['answer.summary.text']
df = pd.DataFrame(data)

# 텍스트 전처리 및 빈도수 계산 함수
def get_top_words_from_column(df, column_name, top_n=10):
    # 전체 텍스트 합치기
    all_text = " ".join(df[column_name].dropna())

    # 단어 토큰화 (어절 단위)
    words = word_tokenize(all_text)

    # 단어 빈도수 계산
    word_counts = Counter(words)

    # 빈도수 높은 상위 N개의 단어 추출
    top_words = word_counts.most_common(top_n)

    return top_words

# answer.summary.text 칼럼에서 상위 10개의 단어와 빈도수 추출
top_words = get_top_words_from_column(df, 'answer.summary.text', 100)
print(top_words)


[('.', 62564), ('수', 19460), (',', 13465), ('저는', 9676), ('있는', 9341), ('있습니다', 8530), ('생각합니다', 7688), ('때문에', 7205), ('이', 7090), ('그', 6931), ('잘', 6801), ('할', 6223), ('것', 5988), ('더', 5766), ('때', 5687), ('제가', 5429), ('일을', 5154), ('가장', 5070), ('하는', 4729), ('많이', 4657), ('같습니다', 4532), ('어떤', 4430), ('하고', 4416), ('것이', 3910), ('것입니다', 3696), ('다른', 3609), ('것을', 3578), ('있고', 3565), ('대한', 3528), ('많은', 3143), ('좋은', 2939), ('합니다', 2775), ('한', 2749), ('그래서', 2743), ('될', 2645), ('있다고', 2495), ('위해', 2369), ('가지고', 2205), ('싶습니다', 2100), ('굉장히', 2064), ('되었습니다', 2059), ('그런', 2056), ('해', 2044), ('있을', 2029), ('하지만', 1986), ('것은', 1983), ('그리고', 1871), ('큰', 1859), ('있기', 1851), ('제', 1798), ('함께', 1775), ('또', 1722), ('했습니다', 1718), ('통해', 1688), ('새로운', 1680), ('업무를', 1661), ('싶은', 1641), ('개발', 1634), ('있어서', 1593), ('생각을', 1589), ('때문입니다', 1536), ('지원하게', 1530), ('다양한', 1523), ('보고', 1503), ('하면서', 1473), ('이런', 1441), ('여러', 1426), ('의견을', 1411), ('해야', 1405), ('공부를', 1405), ('하게', 1394), ('때는', 1391), ('항상', 1381), ('분야에', 1377), ('프로그래밍', 1360), ('업무', 1349), ('알고', 1341), ('서로', 1330), ('먼저', 1327), ('회사에', 1324), ('대해', 1322), ('같은', 1294), ('해서', 1292), ('도움이', 1283), ('있었습니다', 1272), ('그것을', 1257), ('현재', 1241), ('통해서', 1230), ('되고', 1189), ('회사', 1182), ('것이라고', 1164), ('있다는', 1149), ('노력을', 1146), ('좀', 1140), ('시간을', 1128), ('되는', 1124), ('있도록', 1116), ('중', 1116), ('대해서', 1101), ('중요한', 1099)]